## Презентация: Прогнозирование количества поездок желтого такси в Нью-Йорке

Будем использовать встроенные в jupyter notebook виджеты, folium и plotly.offline.

In [1]:
import ipywidgets as wg
from IPython.display import display
import pandas as pd
import numpy as np
#import datetime
import plotly.offline
import folium
import geojson
from plotly.graph_objs import *
import warnings
warnings.filterwarnings('ignore')

plotly.offline.init_notebook_mode(connected=True)

Загружаем факт и прогноз с прошлой недели:

In [2]:
forecast = pd.read_csv('taxi_week6_output1.csv')
data = pd.read_csv('all_cells.csv')
data = data.set_index('start_datetime')
data = data['2016-06-01':]
data.index = data.index.to_datetime()

Преобразуем выходной формат в удобный датафрейм:

In [3]:
forecast['num'], forecast['date'], forecast['hour'], forecast['step'] = zip(*forecast['id'].str.split('_'))
forecast['datetime'] = pd.to_datetime(forecast['date'] + ' ' + forecast['hour'].str.zfill(2))
forecast['forecast_date'] = forecast['datetime'] + pd.to_timedelta(forecast['step'].astype(int), unit = 'h')
forecast.head()

,id,y,num,date,hour,step,datetime,forecast_date
0,1075_2016-05-31_23_1,10.029701,1075,2016-05-31,23,1,2016-05-31 23:00:00,2016-06-01 00:00:00
1,1075_2016-05-31_23_2,0.000000,1075,2016-05-31,23,2,2016-05-31 23:00:00,2016-06-01 01:00:00
2,1075_2016-05-31_23_3,0.000000,1075,2016-05-31,23,3,2016-05-31 23:00:00,2016-06-01 02:00:00
3,1075_2016-05-31_23_4,0.000000,1075,2016-05-31,23,4,2016-05-31 23:00:00,2016-06-01 03:00:00
4,1075_2016-05-31_23_5,0.000000,1075,2016-05-31,23,5,2016-05-31 23:00:00,2016-06-01 04:00:00


Загрузим координаты ячеек, преобразуем их в geojson, напишем функцию для расчета центра ячейки (для простановки маркеров):

In [4]:
regions = pd.read_csv('regions.csv', sep = ';')
def get_geojson(dataframe):
    features = []
    coordinates = []
    for z, w, e, s, n in dataframe.values:
        c = [(w,n), (e,n), (e, s), (w, s), (w, n)]
        coordinates.append([[w,n], [e,n], [e, s], [w, s], [w, n]])
        f = geojson.Feature(geometry = geojson.Polygon([c], fill_color = 'blue', outline_color='red'), name = int(z))
        features.append(f)
    fc = geojson.FeatureCollection(features)
    dump = geojson.dumps(fc)
    return (coordinates, dump)
def get_center(num):
    st = regions[regions['region']==int(num)]
    return [(st['south'] + (st['north'] - st['south'])/2).values[0], (st['west'] + (st['east'] - st['west'])/2).values[0]]

In [5]:
regions1 = regions[regions['region'].isin(forecast['num'].astype(int))]
coordinates = get_geojson(regions1)

Функция для отрисовки карты. На вход принимает диапазоны дней, часов и номер ячейки.

In [6]:
def show_map(day_range, hour_range, cell):
    day_filter = forecast['forecast_date'].dt.day.isin(range(day_range[0], day_range[1]))
    hour_filter = forecast['forecast_date'].dt.hour.isin(range(hour_range[0], hour_range[1]))
    sd = pd.DataFrame(forecast[(forecast['step']=='1') & day_filter & hour_filter].groupby('num')['y'].sum()).reset_index()
    sd['num'] = sd['num'].astype(int)
    map_osm_cells = folium.Map(location=[40.748817, -73.985428], zoom_start=11, tiles = 'OpenStreetMap')
    map_osm_cells.choropleth(geo_str=coordinates[1], data = sd, columns = ['num', 'y'], key_on = 'feature.name', \
                             fill_color = 'YlOrRd', fill_opacity = 0.8, reset = True, line_opacity = 0.,)
    folium.Marker(get_center(cell), popup = str(cell)).add_to(map_osm_cells)
    display(map_osm_cells)

Функция для отрисовки временного ряда: на вход подаем номер ячейки и количество часов назад, которое прогноз был сделан. Реализация выбора временного диапазона лежит на plotly, т.к. это красивее и удобнее.

In [7]:
def show_data(Cell, Step):
    max_y_by_num = max(forecast[forecast['num'] == str(Cell)]['y'].max(), data[str(Cell)].max())
    data_to_show = forecast[(forecast['num'] == str(Cell)) & (forecast['step'] == str(Step))].sort_values(by = 'datetime')
    data_to_show = data_to_show
    data1 = data
    # Рисуем график
    trace1 = Trace(y = data1[str(Cell)], x = data1[str(Cell)].index, name = 'Fact')
    trace2 = Trace(y = data_to_show['y'], x = data_to_show['forecast_date'], name = 'Forecast')
    error = data_to_show.set_index('forecast_date')['y'] - data1[str(Cell)]
    trace3 = Trace(y = error, x = data1[str(Cell)].index, \
                name = 'Forecast Error', fill = 'tozeroy')
    
    fig_data = [trace1, trace2, trace3]
    layout = Layout(
        title = 'Yellow NY Taxi Trip Forecast',
        yaxis = dict(range = [-error.abs().max(), max_y_by_num], title = 'Trips'),
        xaxis = dict(title = 'Date',
                    rangeselector = dict(buttons = list([
                        dict(count = 1, label = 'day', step = 'day'),
                        dict(count = 7, label = 'week', step = 'day'),
                        dict(step = 'all', label = 'all')
                        ])
                        ),
                     rangeslider = dict(),
                     type = 'date'
                    )
    )
    fig = Figure(data = fig_data, layout = layout)
    plotly.offline.iplot(fig)

Собираем функции в одну, чтобы запустить все в одной ячейке:

In [8]:
def show_all(day_range, hour_range, cell, step):
    show_map(day_range, hour_range, cell)
    show_data(cell, step)

In [10]:
wg.interact(show_all, \
         day_range = wg.IntRangeSlider(min = 1, max = 30, value = [1, 30], layout = wg.Layout(width = '500px')), \
         hour_range = wg.IntRangeSlider(min = 0, max = 23, value = [0, 23], layout = wg.Layout(width = '800px')), \
        cell = sorted(set(forecast['num'])), step = wg.IntSlider(min = 1, max = 6))

<function __main__.show_all>